### 1. Time-series extraction

In [1]:
# Config
vars = [
    "PAYEMS", "JTSJOL", "CPIAUCSL", "DGORDER", "HSN1F", "RSAFS", "UNRATE", "HOUST", "INDPRO", "PPIFIS", "DSPIC96", "BOPTEXP", 'BOPTIMP', "WHLSLRIMSA", "TTLCONS", "IR", "CPILFESL", "PCEPILFE", "PCEPI", "PERMIT", "TCU", "BUSINV", "IQ", "GACDISA066MSFRBNY", "PCEC96", "GACDFSA066MSFRBPHI", "GDPC1", "ULCNFB", "A261RX1Q020SBEA"
]
resid_vars = [
    'ICSA', 
    # 'IC4WSA', 
    # 'BUSINV',
    # 'MNFCTRIMSA',
    # 'RETAILIMSA',
    # 'WHLSLRIMSA',
    # 'ISRATIO',
    # 'MNFCTRIRSA',
    # 'RETAILIRSA',
    # 'WHLSLRIRSA',
    # 'CBI',
    # 'CBIC1',
    # A371RX1Q020SBEA,
    # INVCMRMT,
    ]


config = []
for v in vars+resid_vars:
    config.append({
        "source_dataset_code": v,
        "skip_country_codes": [],
        "include_country_codes": []
    })

config

# SQL
# select VariableCode, ReferenceDate, VariableValue, PublicationDate, FrequencyDescription
#     from VintageData vd
#     left join Variable v on v.VariableId=vd.VariableId
#     where VariableCode in (
#         'PAYEMS', 'JTSJOL', 'CPIAUCSL', 'DGORDER', 'HSN1F', 'RSAFS', 'UNRATE', 'HOUST', 'INDPRO', 'PPIFIS', 'DSPIC96', 'BOPTEXP', 'BOPTIMP', 'WHLSLRIMSA', 'TTLCONS', 'IR', 'CPILFESL', 'PCEPILFE', 'PCEPI', 'PERMIT', 'TCU', 'BUSINV', 'IQ', 'GACDISA066MSFRBNY', 'PCEC96', 'GACDFSA066MSFRBPHI', 'GDPC1', 'ULCNFB', 'A261RX1Q020SBEA', 'ICSA', 'ISRATIO', 'CBIC1'
#     )

[{'source_dataset_code': 'PAYEMS',
  'skip_country_codes': [],
  'include_country_codes': []},
 {'source_dataset_code': 'JTSJOL',
  'skip_country_codes': [],
  'include_country_codes': []},
 {'source_dataset_code': 'CPIAUCSL',
  'skip_country_codes': [],
  'include_country_codes': []},
 {'source_dataset_code': 'DGORDER',
  'skip_country_codes': [],
  'include_country_codes': []},
 {'source_dataset_code': 'HSN1F',
  'skip_country_codes': [],
  'include_country_codes': []},
 {'source_dataset_code': 'RSAFS',
  'skip_country_codes': [],
  'include_country_codes': []},
 {'source_dataset_code': 'UNRATE',
  'skip_country_codes': [],
  'include_country_codes': []},
 {'source_dataset_code': 'HOUST',
  'skip_country_codes': [],
  'include_country_codes': []},
 {'source_dataset_code': 'INDPRO',
  'skip_country_codes': [],
  'include_country_codes': []},
 {'source_dataset_code': 'PPIFIS',
  'skip_country_codes': [],
  'include_country_codes': []},
 {'source_dataset_code': 'DSPIC96',
  'skip_countr

In [2]:
len(vars)

29

### 2. Actual real-time vintage preparation

In [3]:
import os
import pandas as pd
import numpy as np

from dateutil import relativedelta

from typing import List

import warnings
warnings.filterwarnings("ignore")

In [4]:
def convert_to_datetime(df: pd.DataFrame, colnames: List[str]) -> pd.DataFrame:
    for col in colnames:
        df[col] = pd.to_datetime(df[col])
    return df

def align_dates(
    dataframe: pd.DataFrame,
    date_colname: str,
) -> pd.DataFrame:
    """Align reference dates to the 1st. days of months."""
    df = dataframe.reset_index(drop=True).reset_index()
    df[date_colname] = df[date_colname].apply(lambda x: x.replace(day=1))
    # mask = (
    #     df[["index", "series_code", "reference_date", "PublicationDate", "VariableValue"]]
    #     .groupby(["series_code", "reference_date", "PublicationDate"])
    #     .first(numeric_only=True)
    # )

    # return pd.merge(df, mask[["index"]], on="index").drop(columns=["index"])
    return df.drop(columns=["index"])


# def pivot_newestdata(
#     dataframe: pd.DataFrame,
#     category_colname: str,
#     date_colname: str,
#     series_colname: str,
# ) -> pd.DataFrame:
#     """Reshape data from long to wide format."""
#     _df_pivot = dataframe.copy()
#     df_pivot = _df_pivot.pivot(
#         columns=category_colname, index=date_colname, values=series_colname
#     )
#     df_pivot = df_pivot.loc[df_pivot.index.is_month_start]

#     df_pivot.index.name = date_colname
#     df_pivot.columns = df_pivot.columns.astype(str)

#     return df_pivot.sort_index()

#### Parameters section

In [5]:
dpath = "./data/vintagedata.csv"
out_path = "./data/0_art_vintages"

target_variable = "GDPC1"
test_yrs_cnt = 6

In [6]:
df = pd.read_csv(dpath, decimal=",")
df = convert_to_datetime(df, ["ReferenceDate", "PublicationDate"])
print(df["FrequencyDescription"].unique())

['Quarterly' 'Monthly' 'Weekly']


In [7]:
df = df.loc[df["ReferenceDate"] < "2023-01-01"]
df = df.loc[df["VariableCode"].isin(vars+resid_vars)]

# publication dates fix
tmp = df.loc[df["VariableCode"] == "GACDISA066MSFRBNY"].copy()
tmp = tmp.drop(columns=["PublicationDate"]).merge(df.loc[df["VariableCode"] == "GACDFSA066MSFRBPHI"][["ReferenceDate", "PublicationDate"]], how="left", on="ReferenceDate")
df = pd.concat([df.loc[df["VariableCode"] != "GACDISA066MSFRBNY"], tmp])

In [8]:
dt = df.loc[df["VariableCode"] == target_variable]["ReferenceDate"].unique()
reference_dates = [d for d in dt if d > dt.max()-relativedelta.relativedelta(years=test_yrs_cnt)]

#### Execution section

In [9]:
i, iters = 0, int(len(reference_dates))
skip_dates = pd.to_datetime(["2019-01-01", "2022-10-01"])

for date in (set(reference_dates)-set(skip_dates)):
    # df_long = pd.DataFrame()
    i = i + 1
    print("Iter " + str(i) + " / " + str(iters))
    
    date=date.strftime("%Y-%m-%d")

    reference_date = pd.to_datetime(date)

    # y data prep.
    y_long = df.loc[df["VariableCode"] == target_variable]
    y_pivot = y_long.pivot(index="PublicationDate", columns="ReferenceDate", values="VariableValue")
    y_pivot = y_pivot.reindex(sorted(pd.date_range(min(y_pivot.columns), max(y_pivot.columns), freq='QS')), axis=1)
    y_pivot = y_pivot.sort_index().ffill()

    pub_dt_limit = y_pivot[reference_date].sort_index().first_valid_index().strftime("%Y-%m-%d")

    # 1. Process all series excluding y
    ## 1.1. Quarterly data prep.
    df_q = align_dates(df.loc[(df["FrequencyDescription"] == "Quarterly") & (df["VariableCode"] != target_variable)], date_colname="ReferenceDate")
    null_cols = []
    df_long_q = pd.DataFrame()

    for variable_code in df_q["VariableCode"].unique():
        series_long = df_q.loc[df_q["VariableCode"] == variable_code]

        series_pivot = series_long.pivot(index="PublicationDate", columns="ReferenceDate", values="VariableValue")
        series_pivot = series_pivot.reindex(sorted(pd.date_range(min(series_pivot.columns), max(series_pivot.columns),freq='QS')), axis=1)
        series_pivot = series_pivot.sort_index().ffill()

        pivot_limit = series_pivot.loc[series_pivot.index < pub_dt_limit]

        # obsługa błędów wyniających z braku obserwacji za zadany okres referencyjny
        try:
            last_release_dt = pivot_limit[reference_date].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
        except (KeyError, AttributeError):
            # print(f"No current release for: {variable_code}, {reference_date}")
            null_cols.append(variable_code)
            try:
                last_release_dt = pivot_limit[reference_date-relativedelta.relativedelta(months=3)].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
            except (KeyError, AttributeError):
                print(f"No former release for: {variable_code}, {reference_date}")
                last_release_dt = None
                continue
        series = pivot_limit.loc[last_release_dt].to_frame()
        series = series.loc[series.first_valid_index():min(series.last_valid_index(), reference_date)]

        series = series.reindex(sorted(pd.date_range(min(series.index), max(series.index), freq='QS')))

        series.columns = ["VariableValue"]
        if series.index.max() == (reference_date - relativedelta.relativedelta(months=3)):
            series["VariableCode"] = f"lag1({variable_code})"
        else:
            series["VariableCode"] = variable_code
        series["PublicationDate"] = last_release_dt

        if series["VariableValue"].isnull().any():
            null_cols.append(variable_code)

        df_long_q = pd.concat([df_long_q, series.reset_index()])

    ## 1.2. Monthly data prep.
    df_m = align_dates(df.loc[(df["FrequencyDescription"] == "Monthly")], date_colname="ReferenceDate")
    df_long_m = pd.DataFrame()
    for variable_code in df_m["VariableCode"].unique():
        series_long = df_m.loc[df_m["VariableCode"] == variable_code]

        series_pivot = series_long.pivot(index="PublicationDate", columns="ReferenceDate", values="VariableValue")
        series_pivot = series_pivot.reindex(sorted(pd.date_range(min(series_pivot.columns), max(series_pivot.columns),freq='MS')), axis=1)
        series_pivot = series_pivot.sort_index().ffill()

        pivot_limit = series_pivot.loc[series_pivot.index < pub_dt_limit]
        dt_mask = pd.DataFrame({"date": pivot_limit.columns, "qs_date": pivot_limit.columns + pd.offsets.QuarterBegin(startingMonth=1)})

        reference_date_m = dt_mask.loc[dt_mask["qs_date"] == (reference_date+relativedelta.relativedelta(months=3))]["date"].max()
        # obsługa błędów wyniających z braku obserwacji za zadany okres referencyjny
        
        try:
            last_release_dt = pivot_limit[reference_date_m].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
        except (KeyError, AttributeError):
            # print(f"No current release for: {variable_code}, {reference_date}")
            null_cols.append(variable_code)
            try:
                last_release_dt = pivot_limit[reference_date_m-relativedelta.relativedelta(months=1)].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
            except (KeyError, AttributeError):
                # print(f"No former release for: {variable_code}, {reference_date}")
                last_release_dt = None
                continue
        
        series = pivot_limit.loc[last_release_dt].to_frame()
        series = series.loc[series.first_valid_index():min(series.last_valid_index(), reference_date_m)]

        series = series.reindex(sorted(pd.date_range(min(series.index), max(series.index), freq='MS')))
        series.columns = ["VariableValue"]
        if series.index.max() == reference_date_m - relativedelta.relativedelta(months=1):
            series["VariableCode"] = f"lag1({variable_code})"
        else:
            series["VariableCode"] = variable_code

        series["PublicationDate"] = last_release_dt

        if series["VariableValue"].isnull().any():
            null_cols.append(variable_code)

        df_long_m = pd.concat([df_long_m, series.reset_index()])

    ## 1.3. Weekly data prep.
    df_w = df.loc[(df["FrequencyDescription"] == "Weekly")]
    df_long_w = pd.DataFrame()
    variable_code="ICSA"

    series_long = df_w.loc[df_w["VariableCode"] == variable_code]

    series_pivot = series_long.pivot(index="PublicationDate", columns="ReferenceDate", values="VariableValue")
    series_pivot = series_pivot.reindex(sorted(pd.date_range(min(series_pivot.columns), max(series_pivot.columns),freq='W-SAT')), axis=1)

    series_pivot = series_pivot.sort_index().ffill()

    pivot_limit = series_pivot.loc[series_pivot.index < pub_dt_limit]
    dt_mask = pd.DataFrame({"date": pivot_limit.columns, "qs_date": pivot_limit.columns + pd.offsets.QuarterBegin(startingMonth=1)})

    reference_date_w = dt_mask.loc[dt_mask["qs_date"] == (reference_date+relativedelta.relativedelta(months=3))]["date"].max()
    # obsługa błędów wyniających z braku obserwacji za zadany okres referencyjny
    try:
        last_release_dt = pivot_limit[reference_date_w].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
    except (KeyError, AttributeError):
        print(f"No current release for: {variable_code}, {reference_date}")
        null_cols.append(variable_code)
        try:
            last_release_dt = pivot_limit[reference_date_w-relativedelta.relativedelta(weeks=1)].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
        except (KeyError, AttributeError):
            print(f"No former release for: {variable_code}, {reference_date}")
            last_release_dt = None
            continue
        
    series = pivot_limit.loc[last_release_dt].to_frame()
    series = series.loc[series.first_valid_index():min(series.last_valid_index(), reference_date_w)]

    series = series.reindex(sorted(pd.date_range(min(series.index), max(series.index), freq='W-SAT')))
    series.columns = ["VariableValue"]
    series["VariableCode"] = variable_code
    series["PublicationDate"] = last_release_dt

    if series["VariableValue"].isnull().any():
        null_cols.append(variable_code)

    df_long_w = series.reset_index()


    X_df_long = pd.concat([df_long_q, df_long_m, df_long_w])

    rt_vintage_dt = y_pivot.loc[y_pivot.index < pub_dt_limit].index.max()
    y_series = y_pivot[y_pivot.columns[y_pivot.columns <= reference_date]].loc[rt_vintage_dt]  # available before the publication of the first estimation of y for a given reference date    
    y_series.loc[reference_date] = y_pivot.loc[pub_dt_limit].loc[reference_date]  # complemented by the first estimation of the value of y for a given reference date
    y_long = y_series.to_frame().reset_index().rename(columns={y_series.name: "VariableValue"})
    y_long["VariableCode"] = target_variable

    y_long['PublicationDate'] = np.where(
        y_long['ReferenceDate'] < reference_date,
        rt_vintage_dt,
        np.where(y_long['ReferenceDate'] == reference_date, pub_dt_limit, None)
    ) 

    assert target_variable not in X_df_long["VariableCode"].unique()

    df_long = pd.concat(
        [
            convert_to_datetime(y_long, ["ReferenceDate", "PublicationDate"]),
            convert_to_datetime(X_df_long, ["ReferenceDate", "PublicationDate"])
        ]
    )

    print(f'Reference Date: {reference_date}, available variables: {df_long["VariableCode"].unique()}, number of variables: {len(df_long["VariableCode"].unique())}')

Iter 1 / 24
Reference Date: 2018-10-01 00:00:00, available variables: ['GDPC1' 'lag1(A261RX1Q020SBEA)' 'lag1(ULCNFB)' 'lag1(BOPTEXP)'
 'lag1(BOPTIMP)' 'CPIAUCSL' 'CPILFESL' 'DGORDER' 'lag1(DSPIC96)' 'HOUST'
 'lag1(HSN1F)' 'IR' 'JTSJOL' 'PAYEMS' 'lag1(PCEC96)' 'lag1(PCEPI)'
 'lag1(PCEPILFE)' 'PERMIT' 'PPIFIS' 'lag1(TTLCONS)' 'UNRATE'
 'lag1(WHLSLRIMSA)' 'TCU' 'RSAFS' 'IQ' 'INDPRO' 'GACDFSA066MSFRBPHI'
 'lag1(BUSINV)' 'GACDISA066MSFRBNY' 'ICSA'], number of variables: 30
Iter 2 / 24
Reference Date: 2020-07-01 00:00:00, available variables: ['GDPC1' 'lag1(A261RX1Q020SBEA)' 'lag1(ULCNFB)' 'lag1(BOPTEXP)'
 'lag1(BOPTIMP)' 'CPIAUCSL' 'CPILFESL' 'DGORDER' 'lag1(DSPIC96)' 'HOUST'
 'HSN1F' 'IR' 'lag1(JTSJOL)' 'PAYEMS' 'lag1(PCEC96)' 'lag1(PCEPI)'
 'lag1(PCEPILFE)' 'PERMIT' 'PPIFIS' 'lag1(TTLCONS)' 'UNRATE'
 'lag1(WHLSLRIMSA)' 'TCU' 'RSAFS' 'IQ' 'INDPRO' 'GACDFSA066MSFRBPHI'
 'lag1(BUSINV)' 'GACDISA066MSFRBNY' 'ICSA'], number of variables: 30
Iter 3 / 24
Reference Date: 2022-07-01 00:00:00, avail

In [10]:
i, iters = 0, int(len(reference_dates))
skip_dates = pd.to_datetime(["2022-10-01"])

for date in (set(reference_dates)-set(skip_dates)):
    # df_long = pd.DataFrame()
    i = i + 1
    print("Iter " + str(i) + " / " + str(iters))
    
    date=date.strftime("%Y-%m-%d")
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    reference_date = pd.to_datetime(date)

    # y data prep.
    y_long = df.loc[df["VariableCode"] == target_variable]
    y_pivot = y_long.pivot(index="PublicationDate", columns="ReferenceDate", values="VariableValue")
    y_pivot = y_pivot.reindex(sorted(pd.date_range(min(y_pivot.columns), max(y_pivot.columns), freq='QS')), axis=1)
    y_pivot = y_pivot.sort_index().ffill()

    pub_dt_limit = y_pivot[reference_date].sort_index().first_valid_index().strftime("%Y-%m-%d")

    # 1. Process all series excluding y
    ## 1.1. Quarterly data prep.
    df_q = align_dates(df.loc[(df["FrequencyDescription"] == "Quarterly") & (df["VariableCode"] != target_variable)], date_colname="ReferenceDate")
    null_cols = []
    df_long_q = pd.DataFrame()

    for variable_code in df_q["VariableCode"].unique():
        series_long = df_q.loc[df_q["VariableCode"] == variable_code]

        series_pivot = series_long.pivot(index="PublicationDate", columns="ReferenceDate", values="VariableValue")
        series_pivot = series_pivot.reindex(sorted(pd.date_range(min(series_pivot.columns), max(series_pivot.columns),freq='QS')), axis=1)
        series_pivot = series_pivot.sort_index().ffill()

        pivot_limit = series_pivot.loc[series_pivot.index < pub_dt_limit]

        # obsługa błędów wyniających z braku obserwacji za zadany okres referencyjny
        try:
            last_release_dt = pivot_limit[reference_date].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
        except (KeyError, AttributeError):
            # print(f"No current release for: {variable_code}, {reference_date}")
            null_cols.append(variable_code)
            try:
                last_release_dt = pivot_limit[reference_date-relativedelta.relativedelta(months=3)].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
            except (KeyError, AttributeError):
                print(f"No former release for: {variable_code}, {reference_date}")
                last_release_dt = None
                continue
        series = pivot_limit.loc[last_release_dt].to_frame()
        series = series.loc[series.first_valid_index():min(series.last_valid_index(), reference_date)]

        series = series.reindex(sorted(pd.date_range(min(series.index), max(series.index), freq='QS')))

        series.columns = ["VariableValue"]
        series["VariableCode"] = variable_code
        series["PublicationDate"] = last_release_dt

        if series["VariableValue"].isnull().any():
            null_cols.append(variable_code)

        df_long_q = pd.concat([df_long_q, series.reset_index()])

    ## 1.2. Monthly data prep.
    df_m = align_dates(df.loc[(df["FrequencyDescription"] == "Monthly")], date_colname="ReferenceDate")
    df_long_m = pd.DataFrame()
    for variable_code in df_m["VariableCode"].unique():
        series_long = df_m.loc[df_m["VariableCode"] == variable_code]

        series_pivot = series_long.pivot(index="PublicationDate", columns="ReferenceDate", values="VariableValue")
        series_pivot = series_pivot.reindex(sorted(pd.date_range(min(series_pivot.columns), max(series_pivot.columns),freq='MS')), axis=1)
        series_pivot = series_pivot.sort_index().ffill()

        pivot_limit = series_pivot.loc[series_pivot.index < pub_dt_limit]
        dt_mask = pd.DataFrame({"date": pivot_limit.columns, "qs_date": pivot_limit.columns + pd.offsets.QuarterBegin(startingMonth=1)})

        reference_date_m = dt_mask.loc[dt_mask["qs_date"] == (reference_date+relativedelta.relativedelta(months=3))]["date"].max()
        # obsługa błędów wyniających z braku obserwacji za zadany okres referencyjny
        
        try:
            last_release_dt = pivot_limit[reference_date_m].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
        except (KeyError, AttributeError):
            # print(f"No current release for: {variable_code}, {reference_date}")
            null_cols.append(variable_code)
            try:
                last_release_dt = pivot_limit[reference_date_m-relativedelta.relativedelta(months=1)].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
            except (KeyError, AttributeError):
                # print(f"No former release for: {variable_code}, {reference_date}")
                last_release_dt = None
                continue
        
        series = pivot_limit.loc[last_release_dt].to_frame()
        series = series.loc[series.first_valid_index():min(series.last_valid_index(), reference_date_m)]

        series = series.reindex(sorted(pd.date_range(min(series.index), max(series.index), freq='MS')))
        series.columns = ["VariableValue"]
        series["VariableCode"] = variable_code

        series["PublicationDate"] = last_release_dt

        if series["VariableValue"].isnull().any():
            null_cols.append(variable_code)

        df_long_m = pd.concat([df_long_m, series.reset_index()])

    ## 1.3. Weekly data prep.
    df_w = df.loc[(df["FrequencyDescription"] == "Weekly")]
    df_long_w = pd.DataFrame()
    variable_code="ICSA"

    series_long = df_w.loc[df_w["VariableCode"] == variable_code]

    series_pivot = series_long.pivot(index="PublicationDate", columns="ReferenceDate", values="VariableValue")
    series_pivot = series_pivot.reindex(sorted(pd.date_range(min(series_pivot.columns), max(series_pivot.columns),freq='W-SAT')), axis=1)

    series_pivot = series_pivot.sort_index().ffill()

    pivot_limit = series_pivot.loc[series_pivot.index < pub_dt_limit]
    dt_mask = pd.DataFrame({"date": pivot_limit.columns, "qs_date": pivot_limit.columns + pd.offsets.QuarterBegin(startingMonth=1)})

    reference_date_w = dt_mask.loc[dt_mask["qs_date"] == (reference_date+relativedelta.relativedelta(months=3))]["date"].max()
    # obsługa błędów wyniających z braku obserwacji za zadany okres referencyjny
    try:
        last_release_dt = pivot_limit[reference_date_w].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
    except (KeyError, AttributeError):
        print(f"No current release for: {variable_code}, {reference_date}")
        null_cols.append(variable_code)
        try:
            last_release_dt = pivot_limit[reference_date_w-relativedelta.relativedelta(weeks=1)].dropna(how="all").sort_index().last_valid_index().strftime("%Y-%m-%d")          
        except (KeyError, AttributeError):
            print(f"No former release for: {variable_code}, {reference_date}")
            last_release_dt = None
            continue
        
    series = pivot_limit.loc[last_release_dt].to_frame()
    series = series.loc[series.first_valid_index():min(series.last_valid_index(), reference_date_w)]

    series = series.reindex(sorted(pd.date_range(min(series.index), max(series.index), freq='W-SAT')))
    series.columns = ["VariableValue"]
    series["VariableCode"] = variable_code
    series["PublicationDate"] = last_release_dt

    if series["VariableValue"].isnull().any():
        null_cols.append(variable_code)

    df_long_w = series.reset_index()


    X_df_long = pd.concat([df_long_q, df_long_m, df_long_w])

    rt_vintage_dt = y_pivot.loc[y_pivot.index < pub_dt_limit].index.max()
    y_series = y_pivot[y_pivot.columns[y_pivot.columns <= reference_date]].loc[rt_vintage_dt]  # available before the publication of the first estimation of y for a given reference date    
    y_series.loc[reference_date] = y_pivot.loc[pub_dt_limit].loc[reference_date]  # complemented by the first estimation of the value of y for a given reference date
    y_long = y_series.to_frame().reset_index().rename(columns={y_series.name: "VariableValue"})
    y_long["VariableCode"] = target_variable

    y_long['PublicationDate'] = np.where(
        y_long['ReferenceDate'] < reference_date,
        rt_vintage_dt,
        np.where(y_long['ReferenceDate'] == reference_date, pub_dt_limit, None)
    ) 

    assert target_variable not in X_df_long["VariableCode"].unique()

    df_long = pd.concat(
        [
            convert_to_datetime(y_long, ["ReferenceDate", "PublicationDate"]),
            convert_to_datetime(X_df_long, ["ReferenceDate", "PublicationDate"])
        ]
    )

    print(f'Reference Date: {reference_date}, available variables: {df_long["VariableCode"].unique()}, number of variables: {len(df_long["VariableCode"].unique())}')
    to_write = df_long[["VariableCode", "ReferenceDate", "VariableValue"]]  # .loc[df_long["VariableCode"].isin(vars)][["VariableCode", "ReferenceDate", "VariableValue"]]
    if out_path:
        to_write.to_csv(os.path.join(out_path, f'{reference_date.strftime("%Y-%m-%d")}.csv'), index=False)
        # df_long.loc[~df_long["series_code"].isin(null_cols)].to_csv(os.path.join(out_path, f'ext_q1out_vintagedata_notnull_{y_code}_{reference_date.strftime("%Y-%m-%d").replace("-", "")}.csv'), index=False)


Iter 1 / 24
Reference Date: 2018-10-01 00:00:00, available variables: ['GDPC1' 'A261RX1Q020SBEA' 'ULCNFB' 'BOPTEXP' 'BOPTIMP' 'CPIAUCSL'
 'CPILFESL' 'DGORDER' 'DSPIC96' 'HOUST' 'HSN1F' 'IR' 'JTSJOL' 'PAYEMS'
 'PCEC96' 'PCEPI' 'PCEPILFE' 'PERMIT' 'PPIFIS' 'TTLCONS' 'UNRATE'
 'WHLSLRIMSA' 'TCU' 'RSAFS' 'IQ' 'INDPRO' 'GACDFSA066MSFRBPHI' 'BUSINV'
 'GACDISA066MSFRBNY' 'ICSA'], number of variables: 30
Iter 2 / 24
Reference Date: 2020-07-01 00:00:00, available variables: ['GDPC1' 'A261RX1Q020SBEA' 'ULCNFB' 'BOPTEXP' 'BOPTIMP' 'CPIAUCSL'
 'CPILFESL' 'DGORDER' 'DSPIC96' 'HOUST' 'HSN1F' 'IR' 'JTSJOL' 'PAYEMS'
 'PCEC96' 'PCEPI' 'PCEPILFE' 'PERMIT' 'PPIFIS' 'TTLCONS' 'UNRATE'
 'WHLSLRIMSA' 'TCU' 'RSAFS' 'IQ' 'INDPRO' 'GACDFSA066MSFRBPHI' 'BUSINV'
 'GACDISA066MSFRBNY' 'ICSA'], number of variables: 30
Iter 3 / 24
Reference Date: 2022-07-01 00:00:00, available variables: ['GDPC1' 'A261RX1Q020SBEA' 'ULCNFB' 'BOPTEXP' 'BOPTIMP' 'CPIAUCSL'
 'CPILFESL' 'DGORDER' 'DSPIC96' 'HOUST' 'HSN1F' 'IR' 'JTSJOL' '